# Pandas基础

该部分主要参考[Pandas中文网](https://www.pypandas.cn/docs/)和[官方用户手册](https://pandas.pydata.org/pandas-docs/stable/index.html)。

Pandas是一个开源的，为Python编程语言提供高性能，易于使用的**数据结构**和**数据分析工具**。

因此这部分的记录主要思路就是围绕数据结构和分析工具两方面来展开。两部分之间会有重复的地方，就当是复习了。

首先，简单记录下关于pandas的介绍。Pandas适合处理多种不同类型的数据：

- 不同类型的数据列组成的表格数据；
- 有序或无序的时间序列数据；
- 带行列标签的类型相同或不同的行或列组成的矩阵数据；
- 任意形式的观测或统计数据，它们不需标记即可放入pandas数据结构中。 

其运算都主要都围绕两类数据结构展开，Series和Dataframe，接下来从它们开始记录。

Series(1维)和DataFrame(2维)能处理绝大多数的典型应用，包括金融、统计、社科、工程等各个领域。
 
对于R用户，DataFrame提供了所有R的data.frame提供的，并且还有更多。

pandas建立在NumPy之上，也整合了很多第三方库，打造了良好的科学计算环境。

pandas处理的经典问题包括且不限于：

- 处理缺失数据，用NaN表示；
- 数据结构大小可变：可以插入删除DataFrame或者更高维对象的列；
- 自动且明确的data alignment，可以明确指定数据到一系列标签下；
- 灵活数据分组，很容易聚合转换数据；
- 容易将杂乱、标签复杂的Python和NumPy数据结构转换为DataFrame对象；
- 根据标签容易slicing,indexing和subsetting大数据集；
- 容易合并和连接数据集；
- 灵活地reshaping和pivoting数据集；
- 多级标签；
- 从各类文件中加载数据, 包括HDF5格式的数据；
- 时间序列指定的功能：日期范围生成和频率转换,滑动窗口的统计, 滑动线性回归, 日期平移或滞后等。

在数据处理的各个环节，包括变换和清洗数据，分析建模，组织分析结果为正式的格式，可视化和输出数据等，pandas都能很好地处理。

## Pandas数据结构

最好的理解Pandas数据结构的方式是将其当做其更低维数据的容器，比如Series是标量数据的容器，Dataframe是Series的容器。
可以通过类似字典的方式从容器中插入或移除对象。

pandas中为了更直观地给出数据结构不同维度的信息，更多用index和columns，而不是使用axis0和1这样来表示。例如：

``` python
for col in df.columns:
    series = df[col]
    # do something with series
```

pandas所有数据结构都是值可变的，但不总是size可变的。比如Series的长度是不可变的，但是Dataframe中是可以插入列的。
绝大多数方法都会产生新的对象，而不改变原输入数据。一般来说，pandas喜欢保持不可变性。

### Series与Dataframe

对于pandas的这两个数据结构，个人认为从增删改查的角度了解较好，首先是数据结构的初始化，然后增删操作，包括拼接操作等，接下来就是修改名称等，而后重点关注索引切片等查询操作，最后再补充一些分组等其他内容。

#### 初始化

基本上来说，是利用numpy的数组为值依据，用columns和index给列和行起名。

In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
# 创建一个空的 DataFrame
df_empty = pd.DataFrame(columns=['A', 'B', 'C', 'D'])
print(df_empty)
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
print(df)
# 使用numpy数组，命名行列进行初始化
df2 = pd.DataFrame(np.arange(16).reshape((4,4)), columns=['one', 'two', 'three', 'four'], index=['a', 'b', 'c','d'])
print(df2)
# 带时间序列的dataframe的初始化
time_range = pd.date_range('2011-1-1', periods=4, freq='D')
df2.index = time_range
print(df2)
time_range = pd.date_range('2011-1-1', '2011-1-4', freq='D')
df2.columns = time_range
print(df2)

Empty DataFrame
Columns: [A, B, C, D]
Index: []
   a  b  c
0  0  0  0
1  1  1  1
2  2  2  2
   one  two  three  four
a    0    1      2     3
b    4    5      6     7
c    8    9     10    11
d   12   13     14    15
            one  two  three  four
2011-01-01    0    1      2     3
2011-01-02    4    5      6     7
2011-01-03    8    9     10    11
2011-01-04   12   13     14    15
            2011-01-01  2011-01-02  2011-01-03  2011-01-04
2011-01-01           0           1           2           3
2011-01-02           4           5           6           7
2011-01-03           8           9          10          11
2011-01-04          12          13          14          15


和python基础，numpy一样，pandas也有自己的日期处理包，日期总是比较麻烦的。

In [12]:
# 日期／字符串转换
strtime=['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31']
index=pd.to_datetime(strtime)
time=pd.to_datetime(strtime)
data=list(range(1,11))
df=pd.DataFrame([time,data],columns=index,index=['time','data']).T

#索引和time字段均为Timestamp格式数据
print(type(index[0]))

print(type(df.index[0]))

print(type(df.time[0]))


#先来看索引
mt_time=df.index.strftime('%Y-%m')
mt_time

print(type(mt_time[0]))

#针对列（直接操作还不清楚怎么操作）
time_list=[x.strftime('%Y-%m') for x in df['time']]
print(time_list)
#再按月份求和
df['data'].groupby(time_list).sum()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>
['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10']


2000-01     1
2000-02     2
2000-03     3
2000-04     4
2000-05     5
2000-06     6
2000-07     7
2000-08     8
2000-09     9
2000-10    10
Name: data, dtype: int64

#### 增删，拼接等操作

可参考：[Merge, join, and concatenate](https://www.pypandas.cn/docs/user_guide/merging.html#concatenating-objects)

首先是纵向拼接。

In [10]:
import pandas as pd
df0 = pd.DataFrame()
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                   'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                     index=[0, 1, 2, 3]) 

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                   'B': ['B4', 'B5', 'B6', 'B7'],
                  'C': ['C4', 'C5', 'C6', 'C7'],
                     'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7]) 

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                     'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11]) 

frames = [df0, df1, df2, df3]
result = pd.concat(frames)
print(result)

      A    B    C    D
0    A0   B0   C0   D0
1    A1   B1   C1   D1
2    A2   B2   C2   D2
3    A3   B3   C3   D3
4    A4   B4   C4   D4
5    A5   B5   C5   D5
6    A6   B6   C6   D6
7    A7   B7   C7   D7
8    A8   B8   C8   D8
9    A9   B9   C9   D9
10  A10  B10  C10  D10
11  A11  B11  C11  D11


一个多列拼接成一列的小例子

In [8]:
"""几列字符串拼为一列字符串"""
# 结构很简单: 第一列的名称.str.cat(第二列的名称)
df['a'] = df.iloc[:, 0].apply(str) + "-" + df['b'].apply(str) + "-" + df['b'].apply(str)
# 拼接之后，只留下特定的几列
df = df[['a', 'c']]
print(df)

       a  c
0  0-0-0  0
1  1-1-1  1
2  2-2-2  2


多个Series拼接

In [9]:
"""多个Series拼接"""
# 初始化的时候不起名，后面rename没有用
a = pd.Series([1, 2], name='aa')
rng1 = pd.date_range('2011-1-1', periods=2, freq='D')
a.index = rng1
print(a)
b = pd.Series([2, 3, 4])
rng2 = pd.date_range('2011-1-2', periods=3, freq='D')
b.index = rng2
c = pd.Series([5, 6])
rng3 = pd.date_range('2011-1-1', periods=2, freq='D')
c.index = rng3
series1 = pd.concat([a, b], axis=1)
# 拼接时，给各列取名
print(series1)
df2 = pd.concat([series1, c], axis=1)
print(df2)

2011-01-01    1
2011-01-02    2
Freq: D, Name: aa, dtype: int64
             aa    0
2011-01-01  1.0  NaN
2011-01-02  2.0  2.0
2011-01-03  NaN  3.0
2011-01-04  NaN  4.0
             aa    0    0
2011-01-01  1.0  NaN  5.0
2011-01-02  2.0  2.0  6.0
2011-01-03  NaN  3.0  NaN
2011-01-04  NaN  4.0  NaN


#### 修改操作

包括修改行列名，以及交换行列等操作。

In [ ]:
"""重新命名各列"""
new_col = ['new1', 'new2', 'new3', 'new4']
df2.columns = new_col
print(df2)

"""交换列的位置"""
order = ['new2', 'new1', 'new3', 'new4']
df2 = df2[order]
print(df2)

行列拆分，参考[DataFrame一列拆成多列以及一行拆成多行](https://blog.csdn.net/Asher117/article/details/84346073)，在数据分析时，经常需要把DataFrame的一列拆成多列或者根据某列把一行拆成多行。

In [5]:
import pandas as pd

df= pd.DataFrame({'Country': ['China', 'America', 'Japan'],
                   'City': ['Shanghai|Shenzhen', 'New York|State College', 'Tokyo|Osaka']},
                     index=[0, 1, 2])
df

,Country,City
0,China,Shanghai|Shenzhen
1,America,New York|State College
2,Japan,Tokyo|Osaka


In [6]:
df['Citys']=df['City'].map(lambda x: x.split('|'))
df

,Country,City,Citys
0,China,Shanghai|Shenzhen,"[Shanghai, Shenzhen]"
1,America,New York|State College,"[New York, State College]"
2,Japan,Tokyo|Osaka,"[Tokyo, Osaka]"


In [7]:
df['City1']=df['City'].map(lambda x: x.split('|')[0])
df['City2']=df['City'].map(lambda x: x.split('|')[1])
df

,Country,City,Citys,City1,City2
0,China,Shanghai|Shenzhen,"[Shanghai, Shenzhen]",Shanghai,Shenzhen
1,America,New York|State College,"[New York, State College]",New York,State College
2,Japan,Tokyo|Osaka,"[Tokyo, Osaka]",Tokyo,Osaka


再比如，常见的日期拆分，

#### 索引，切片等操作

首先是直接[]索引，以及loc和iloc的使用，这是pandas中最常用的索引方式。

In [16]:
import numpy as np
import pandas as pd

ser = pd.Series(np.arange(3.))

data = pd.DataFrame(np.arange(16).reshape(4,4),index=list('abcd'),columns=list('wxyz'))

data['w']  #选择表格中的'w'列，使用类字典属性,返回的是Series类型

data.w    #选择表格中的'w'列，使用点属性,返回的是Series类型

data[['w']]  #选择表格中的'w'列，返回的是DataFrame属性

data[['w','z']]  #选择表格中的'w'、'z'列

data[0:2]  #返回第1行到第2行的所有行，前闭后开，包括前不包括后

data[1:2]  #返回第2行，从0计，返回的是单行，通过有前后值的索引形式，
       #如果采用data[1]则报错

data.iloc[1:2] #返回第2行的第三种方法，返回的是DataFrame，跟data[1:2]同

data['a':'b']  #利用index值进行切片，返回的是**前闭后闭**的DataFrame, 

data.head()  #返回data的前几行数据，默认为前五行，需要前十行则dta.head(10)
data.tail()  #返回data的后几行数据，默认为后五行，需要后十行则data.tail(10)


data.iloc[-1]   #选取DataFrame最后一行，返回的是Series
data.iloc[-1:]   #选取DataFrame最后一行，返回的是DataFrame

data.loc['a',['w','x']]   #返回‘a’行'w'、'x'列，这种用于选取行索引列索引已知

data.iat[1,1]   #选取第二行第二列，用于已知行、列位置的选取。

5

In [7]:
"""取dataframe指定多行列 slice操作"""
# 取多行
print(df2.iloc[0:2])
# 取多列
print(df2.iloc[:, 0:2])
# 取多行多列
print(df2.iloc[0:2, 0:2])
# 行用数字取，列用名字取
print(df2.iloc[0:2]['new1'])

"""按日期取值并处理"""
print(df2)
print(type(df2.index))
print('---------获取2011年前两天的数据-----------')
# 获取某段日期内的数据
print(df2['2011-01-01':'2011-01-02'])


            new1  new2  new3  new4
2011-01-01     0     1     2     3
2011-01-02     4     5     6     7
2011-01-03     8     9    10    11
2011-01-04    12    13    14    15
            new2  new1  new3  new4
2011-01-01     1     0     2     3
2011-01-02     5     4     6     7
2011-01-03     9     8    10    11
2011-01-04    13    12    14    15
            new2  new1  new3  new4
2011-01-01     1     0     2     3
2011-01-02     5     4     6     7
            new2  new1
2011-01-01     1     0
2011-01-02     5     4
2011-01-03     9     8
2011-01-04    13    12
            new2  new1
2011-01-01     1     0
2011-01-02     5     4
2011-01-01    0
2011-01-02    4
Freq: D, Name: new1, dtype: int32
            new2  new1  new3  new4
2011-01-01     1     0     2     3
2011-01-02     5     4     6     7
2011-01-03     9     8    10    11
2011-01-04    13    12    14    15
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
---------获取2011年前两天的数据-----------
            new2  new1  new3  ne

series的slice

In [3]:
"""给Series作slice操作"""
arr = [1, 2, 3, 4]  # 创建数组
series_1 = pd.Series(arr)
series_1.index = ['a', 'b', 'c', 'd']
print("------------------Series查询操作----------------------")
print(series_1['a'])
print(series_1[['a', 'b']])
print(series_1[series_1 > 2])
print(series_1[:2])
print(series_1['a':'c'])

------------------Series查询操作----------------------
1
a    1
b    2
dtype: int64
c    3
d    4
dtype: int64
a    1
b    2
dtype: int64
a    1
b    2
c    3
dtype: int64


在dataframe中还常用到条件筛选。参考[30分钟带你入门数据分析工具 Pandas（上篇），果断收藏](https://zhuanlan.zhihu.com/p/44174554)，用中括号 [] 的方式，除了直接指定选中某些列外，还能接收一个条件语句，然后筛选出符合条件的行/列。比如：

In [1]:
import pandas as pd
import numpy as np
df=pd.DataFrame(np.random.randn(5,4),['A','B','C','D','E'],['W','X','Y','Z'])
# 筛选出 'W'>0 的行
print(df[df['W']>0])
print('\n')
# 只看 'X' 列中 'W'>0 的数据
print(df[df['W']>0]['X'])
print('\n')
print(df[df['W']>0][['X','Y']])

          W         X         Y         Z
C  0.479199  1.551477  1.305261 -0.191563
D  1.388126  0.176622 -0.475049 -1.042992


C    1.551477
D    0.176622
Name: X, dtype: float64


          X         Y
C  1.551477  1.305261
D  0.176622 -0.475049


还可以用逻辑运算符 &（与）和 |（或）来链接多个条件语句，以便一次应用多个筛选条件到当前的 DataFrame 上，比如筛选出同时满足 'W'>0 和'X'>1 的行：

In [2]:
df[(df['W']>0) & (df['X']>1)]

,W,X,Y,Z
C,0.479199,1.551477,1.305261,-0.191563


再比如，还可以让列之间进行比较并作为索引条件

In [4]:
df

,W,X,Y,Z
A,-2.003236,0.294592,-0.626207,0.711485
B,-0.744839,-0.517342,-0.276921,0.975167
C,0.479199,1.551477,1.305261,-0.191563
D,1.388126,0.176622,-0.475049,-1.042992
E,-0.484756,1.224973,0.043494,0.814796


In [6]:
df[df['W']<df['X']]

,W,X,Y,Z
A,-2.003236,0.294592,-0.626207,0.711485
B,-0.744839,-0.517342,-0.276921,0.975167
C,0.479199,1.551477,1.305261,-0.191563
E,-0.484756,1.224973,0.043494,0.814796


#### 分组

对dataframe进行分组的例子。

In [5]:
salaries = pd.DataFrame({
    'name': ['BOSS', 'Lilei', 'Lilei', 'Han', 'BOSS', 'BOSS', 'Han', 'BOSS'],
    'Year': [2016, 2016, 2016, 2016, 2017, 2017, 2017, 2017],
    'Salary': [999999, 20000, 25000, 3000, 9999999, 999999, 3500, 999999],
    'Bonus': [100000, 20000, 20000, 5000, 200000, 300000, 3000, 400000]
})
print(salaries.columns)
print(salaries.info())
print(salaries.describe())
salaries = salaries[['name', 'Year', 'Salary', 'Bonus']]
# 定顺序
print(salaries)
# 对dataframe按name进行分组
group_by_name = salaries.groupby('name')
# 获取分组后的某一组
se_temp = group_by_name.get_group('Lilei')
print(se_temp)
print(se_temp.describe())
# 循环各组，并将名字在给定的序列中的group添加到数组中
config = pd.Series({'names': ['Lilei', 'BOSS']})
dfs = []
for name, group in group_by_name:
    print(name)
    if name in config['names']:
        dfs.append(group)

for i in range(len(dfs)):
    # 如果没有把name和group分开，那么dfs[i]的类型会是tuple，key为name，value是group，可参考接下来的输出
    print(type(dfs[i]))
    print(dfs[i])

# 如果没有把name和group分开，那么dfs[i]的类型会是tuple，key为name，value是group
dfs_s = []
for group in group_by_name:
    dfs_s.append(group)
for i in range(len(dfs_s)):
    print(type(dfs_s[i]))
    print(dfs_s[i])

Index(['name', 'Year', 'Salary', 'Bonus'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
name      8 non-null object
Year      8 non-null int64
Salary    8 non-null int64
Bonus     8 non-null int64
dtypes: int64(3), object(1)
memory usage: 384.0+ bytes
None
              Year        Salary          Bonus
count     8.000000  8.000000e+00       8.000000
mean   2016.500000  1.631437e+06  131000.000000
std       0.534522  3.416521e+06  152851.935826
min    2016.000000  3.000000e+03    3000.000000
25%    2016.000000  1.587500e+04   16250.000000
50%    2016.500000  5.124995e+05   60000.000000
75%    2017.000000  9.999990e+05  225000.000000
max    2017.000000  9.999999e+06  400000.000000
    name  Year   Salary   Bonus
0   BOSS  2016   999999  100000
1  Lilei  2016    20000   20000
2  Lilei  2016    25000   20000
3    Han  2016     3000    5000
4   BOSS  2017  9999999  200000
5   BOSS  2017   999999  300000
6    Han  2017  

### pandas与numpy数据结构之间的转换

pandas转为numpy比较简单，直接调用value即转为ndarray。反过来，也很容易，直接用pandas的类型，相当于直接初始化了。

In [21]:
# pandas dataframe与numpy array之间的转换
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
print(df)
# df转换为ndarray
array_from_df=df.values
print(array_from_df)
print(np.array(df))
# 读取某一列数据，两种方式均可
print(df['A'])
print(df.loc[:, 'A'])
# 一列数据转换为ndarray
print(np.array(df['A']))
# Series转ndarray
# Creating the Series
sr = pd.Series(['New York', 'Chicago', 'Toronto', 'Lisbon', 'Rio'])
# Create the Index
index_ = ['City 1', 'City 2', 'City 3', 'City 4', 'City 5']
# set the index
sr.index = index_
# return numpy array representation
result = sr.values
# Print the result
print("series转为ndarray：")
print(result)
# Print the series
print(sr)

   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9
[[1 4 7]
 [2 5 8]
 [3 6 9]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]
0    1
1    2
2    3
Name: A, dtype: int64
0    1
1    2
2    3
Name: A, dtype: int64
[1 2 3]
series转为ndarray：
['New York' 'Chicago' 'Toronto' 'Lisbon' 'Rio']
City 1    New York
City 2     Chicago
City 3     Toronto
City 4      Lisbon
City 5         Rio
dtype: object


## Pandas数据分析

从数据读取、处理、分析和可视化及输出几个方面展开。

### IO工具

对于csv文件和txt文件，都可以通过pandas的read_csv()函数读取。

In [5]:
from io import StringIO, BytesIO

data = ('col1,col2,col3\n'
         'a,b,01\n'
         'a,b,02\n'
         'c,d,03')
print(data)
print(pd.read_csv(StringIO(data)))
# 读取数据的时候即进行slice
print(pd.read_csv(StringIO(data), usecols=range(0,2)))
print(pd.read_csv(StringIO(data), usecols=lambda x: x.upper() in ['COL1', 'COL3']))
print(pd.read_csv(StringIO(data), skiprows=lambda x: x % 2 != 0))

col1,col2,col3
a,b,01
a,b,02
c,d,03
  col1 col2  col3
0    a    b     1
1    a    b     2
2    c    d     3
  col1 col2
0    a    b
1    a    b
2    c    d
  col1  col3
0    a     1
1    a     2
2    c     3
  col1 col2  col3
0    a    b     2


"""专业方面，很多数据都是把年月或者年旬或者年日分别当做行和列，在计算时，需要进行处理，把时间变为一列或一行，即把几列或行的数组拼接起来
即将[1 2;2 3]的数据变为[1 2 2 3]，pandas的concat和numpy的concatenate类似。最后把行名index统一换成日期，构成时间序列Series"""

In [6]:
# 如果遇到“UnicodeDecodeError: 'utf8' codec can't decode byte....”错误，用记事本另存csv文件时，将“编码”设置为‘UTF-8’即可
dataset = pd.read_csv('Sheet1.csv')
print(dataset)
# header指定某行的值作为各列的列名，1表示第二行，如果是None，则从第一行开始就是数据。
dataset = pd.read_csv('Sheet1.csv', header=1)
print(dataset)
# 因为给定的数据格式，各人有各自的一套，所以具体情况具体分析。比如，这里dataset的第一列可以去掉，即得到所有数据，删除列时，用drop函数，加参数axis=1，不加则表示删除行
dataset1 = dataset.drop(['旬平均'], axis=1)
print(dataset1)
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
# iloc获取的数据格式为Series
se = df.iloc[:, 0]
# 获取dataframe的列数：df.shape[1]
for i in range(1, df.shape[1]):
    print("拼接第" + str(i) + "列:")
    se_temp = df.iloc[:, i]
    se = pd.concat([se, se_temp])
    print(se)
# 如果一开始没有给series起名，比如从csv读取出来时是dataframe，拼接之后没办法再给Series起名，但是又需要给它起名，那么只能采取重新构造一个series的方式来命名
se = pd.Series(se, name="aaaaaaa")
print(se)
# 把每行名称换为日期
print("index换为日期：")
rng = pd.date_range('2011-1-1', periods=9, freq='H')
print(rng)
se.index = rng
se = pd.Series(se, name="bbbbb")
print(se)

                                     旬流量   单位:m3/s
旬平均   1970 1971 1972 1973 1974 1975           1976
1月上旬  946  958  1030 900  861  1060            975
1月中旬  895  908  969  895  885  1030            890
1月下旬  891  947  891  916  868  958             885
2月上旬  887  1050 933  885  993  967             914
2月中旬  919  1060 991  926  927  1030            909
2月下旬  1060 1150 1120 967  1100 1150           1090
3月上旬  1040 1150 967  949  937  1260            974
3月中旬  1160 1260 1230 961  980  1340           1130
3月下旬  1210 1110 1170 973  1080 1300           1100
4月上旬  1410 1180 1330 1350 1730 1500           1420
4月中旬  1770 1370 1500 1440 1840 1330           1790
4月下旬  1700 1330 1870 1160 1740 1390           1610
5月上旬  2510 2030 2020 2230 2410 2180           2290
5月中旬  4460 1870 2350 2680 2510 2440           2100
5月下旬  2720 2690 2950 2670 2860 2480           3240
6月上旬  3030 3400 3030 3590 2700 2380           3560
6月中旬  2300 3610 3250 6400 3280 3760           3470
6月下旬  2860 5380 4390 6100 5320 

当csv中包含了大量的编号代码，比如是002开头的编号——002111， 使用pd.read_csv('text.csv') 则会让所有的002xxx，变成了2xxx，前面2个0不见了，因为作为数值类型，没有必要保留0。

因此，为了保持编号的字符串特性，直接在参数一栏设置一下即可：
df＝pd.read_csv('text.csv', dtype={'code':str}
 
这样，把你要转换的列的名字设定好， “code”列中的数据读取为str，用列名或者列序号均可。
 
这样，读取到的数据就是按照我们的要求的了。

In [7]:
dataset = pd.read_csv(StringIO(data),dtype={2:str})
print(dataset)
dataset = pd.read_csv(StringIO(data),dtype={'col3':str})
print(dataset)

  col1 col2 col3
0    a    b   01
1    a    b   02
2    c    d   03
  col1 col2 col3
0    a    b   01
1    a    b   02
2    c    d   03



### Prepare Data

#### 修改名称

包括列名和行名，行一般又称index；列名也称field。

总体上可以分为在读数据时修改和读后修改两种。

具体的，行的修改方法有多种，参考：[重命名dataframe的index](https://blog.csdn.net/sinat_35930259/article/details/79872577)；列名的修改方法也有多种，参考[Pandas中修改DataFrame列名](http://www.voidcn.com/article/p-wycobfgd-bqs.html)

In [8]:
# 给出所有列名，这里类型是Index，可以直接转为list
print(dataset.columns)
columns=dataset.columns.tolist()
print(columns)
# 转换后，每个元素直接就是字符串了
print(type(columns[0]))
# 修改列名的常用方式,inplace为false的话，还需要赋值到dataset，因此直接设置为true
dataset.rename(columns={'col1':'a', 'col2':'b', 'col3':'c'}, inplace = True)
print(dataset)
# 修改index行名

Index(['col1', 'col2', 'col3'], dtype='object')
['col1', 'col2', 'col3']
<class 'str'>
   a  b   c
0  a  b  01
1  a  b  02
2  c  d  03


#### Slice

切片包括横向切，纵向切，即选取某些行，选取某些列的操作。dataframe的切片操作很容易，记住灵活运用其index的功能即可，第一维是行，第二维是列，定位使用loc，数值定位用iloc。

In [9]:
# 取行直接用index即可
print(dataset.loc[0:1,:])
print("注意因为行名是0-1-2，所以loc和iloc略有不同")
print(dataset.iloc[0:1,:])
# 取列用field
print("列操作")
print(dataset.loc[:,'c'])
columns=['a','c']
print(dataset.loc[:,columns])

   a  b   c
0  a  b  01
1  a  b  02
注意因为行名是0-1-2，所以loc和iloc略有不同
   a  b   c
0  a  b  01
列操作
0    01
1    02
2    03
Name: c, dtype: object
   a   c
0  a  01
1  a  02
2  c  03


#### 数据类型转换

参考[Pandas数据类型转换的几个小技巧](https://zhuanlan.zhihu.com/p/35287822)

Pandas中进行数据类型转换有三种基本方法：

- 使用astype()函数进行强制类型转换
- 自定义函数进行数据类型转换
- 使用Pandas提供的函数如to_numeric()、to_datetime()

In [8]:
dataset['col3'].astype('float')

0    1.0
1    2.0
2    3.0
Name: col3, dtype: float64

当待转换列中含有不能转换的特殊值时(例如￥,ErrorValue,14n等)astype()函数将失效。
astype()函数有效的情形：

- 数据列中的每一个单位都能简单的解释为数字(2, 2.12等）
- 数据列中的每一个单位都是数值类型且向字符串object类型转换

Pandas的astype()函数和复杂的自定函数之间有一个中间段，那就是Pandas的一些辅助函数。这些辅助函数对于某些特定数据类型的转换非常有用(如to_numeric()、to_datetime())。

#### 数据操作

包括旋转、连接等。

In [11]:
# 返回列数：
print(dataset.shape[1])
# 返回行数：
print(dataset.shape[0])
# factorize(values[, sort, order, …])	Encode the object as an enumerated type or categorical variable.
labels, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b'])
print(labels)
uniques

3
3
[0 0 1 2 0]


array(['b', 'a', 'c'], dtype=object)

对dataframe数据执行批量运算，使用pandas的apply函数。

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
print(df)
df.apply(np.sqrt)

   A  B
0  4  9
1  4  9
2  4  9


,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


In [5]:
# 作用到每列
df.apply(np.sum, axis=0)

A    12
B    27
dtype: int64

In [6]:
# 作用到每行
df.apply(lambda x: [1, 2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object

- [ ] Analyze Data

### Pandas绘图

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
print(ts)
pyplot.plot(ts.index, ts.values, color='red', label='testing accuracy')
pyplot.show()